# Given someone with suicidal desire, detect active rescue 

Author: Daniel Low (Harvard University)

In [ ]:
# config
location = 'local'
random_seed = 123
# Classification config
toy = False
dvs = ['dv'] #columns which will be DVs

cv = 5
classes = {
	'dv':['suicidal_desire', 'active_rescue'] # 0 and 1
		   }


In [ ]:
!python --version # tested with Python 3.10.12

In [ ]:
# !pip install -q deplacy==2.0.5
# !pip install -q flair==0.13.0
# !pip install -q --upgrade urllib3==2.0.7
# !pip install -q sentence-transformers==2.2.2


In [ ]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import sys
import os
import dill
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from importlib import reload
pd.set_option("display.max_columns", None)
# pd.options.display.width = 0

# local scripts
import srl_constructs
sys.path.append('./../concept-tracker') # wherever cts.py is
from concept_tracker.utils import pipelines, hyperparameters




In [ ]:
# config
np.random.seed(random_seed)

ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')



if location == 'openmind':
  input_dir = '/nese/mit/group/sig/projects/dlow/ctl/datasets/'
  output_dir = 'home/dlow/zero_shot/data/output/'
elif location =='local':
  input_dir = './data/output/ctl/'
  output_dir = './data/output/active_rescue/'
os.makedirs(output_dir, exist_ok=True)



In [ ]:
feature_names = srl_constructs.constructs_in_order

remove_constructs = [
				'Bullying', # 'tell me to kill myself' matches with 'kill myself'
					 'Social withdrawal',  #'want to be alone' matches with many loneliness comments
					 'Suicide exposure' #'suicide aftermath' matches with 'suicide' etc
					 'Agitated',# matches 'frustrated', 'stress'
					 'Emotional pain & psychache', 
					 'Grief & bereavement', #'commited suicide' with 'suicide'
					 'Perfectionism',
					 'Discrimination',#"treated with less respect" matches with common phrase by therapists "No one deserves to be treated that way"
					 ]


# add_constructs = ['texter_word_count',	'counselor_word_count', '1p_sg', '3rd_sg_pl', 'past', 'present','future', 'simple', 'progressive']
# add_constructs = ['amount_of_messages_texter',	'amount_of_messages_counselor']# 'amount_of_messages']#, '1p_sg', '3rd_sg_pl', 'past', 'present','future', 'simple', 'progressive']

add_constructs = ['word_count', 'word_count_texter', 'word_count_counselor', 'word_count_texter_counselor_ratio', 'amount_of_messages_texter', 'amount_of_messages_counselor', 'amount_of_messages_texter_counselor_ratio']





feature_names = [x for x in feature_names if x not in remove_constructs]
feature_names+= add_constructs

feature_vectors = [f'cts_{len(feature_names)}_srl_prototypes']




In [ ]:
train_subset = pd.read_csv('./data/input/ctl/X_train_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes.csv')
test_subset = pd.read_csv('./data/input/ctl/X_test_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes_clauses-all.csv')
test_subset_25 = pd.read_csv('./data/input/ctl/X_test_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes_clauses-46.csv')
test_subset_50 = pd.read_csv('./data/input/ctl/X_test_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes_clauses-71.csv')
test_subset_75 = pd.read_csv('./data/input/ctl/X_test_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes_clauses-106.csv')





In [ ]:
train_subset.shape

In [ ]:
1750*2

In [ ]:
display(train_subset['dv'].value_counts())
display(test_subset['dv'].value_counts(normalize=True))


In [ ]:
# df = train_subset.copy()
# amount_of_messages = 1000

# messages = [n.split('\n')[:amount_of_messages] for n in df[f'message_with_interaction_clean'].values]
# messages_texter_all = []
# messages_counselor_all = []
# word_count_texter = []
# word_count_counselor = []
# word_count_both = []
# for convo in messages:
# 	messages_texter = [n.replace('texter: ','') for n in convo if 'texter:' in n]
# 	messages_texter_all.append(len(messages_texter))
# 	messages_texter_join = ' '.join(messages_texter)
# 	word_count_texter.append(len(messages_texter_join.split()))

# 	messages_counselor = [n.replace('counselor: ','') for n in convo if 'counselor:' in n]
# 	messages_counselor_all.append(len(messages_counselor))
# 	messages_counselor_join = ' '.join(messages_counselor)
# 	word_count_counselor.append(len(messages_counselor_join.split()))
	
# 	convo_join = ' '.join(convo).replace('texter: ', '').replace('counselor: ', '')
# 	word_count_both.append(len(convo_join.split()))




# df[f'word_count'] = word_count_both
# df[f'word_count_texter'] = word_count_texter
# df[f'word_count_counselor'] = word_count_counselor
# df[f'word_count_texter_counselor_ratio'] = df[f'word_count_texter'] / df[f'word_count_counselor']
# df[f'amount_of_messages_texter'] = messages_texter_all
# df[f'amount_of_messages_counselor'] = messages_counselor_all
# df[f'amount_of_messages_texter_counselor_ratio'] = df[f'amount_of_messages_texter'] / df[f'amount_of_messages_counselor']


In [ ]:
messages_25, messages_50, messages_75 = 28,43,64



def word_count_by_person(df, amount_of_messages):
	messages = [n.split('\n')[:amount_of_messages] for n in df[f'message_with_interaction_clean'].values]
	messages_texter_all = []
	messages_counselor_all = []
	word_count_texter = []
	word_count_counselor = []
	word_count_both = []
	for convo in messages:
		messages_texter = [n.replace('texter: ','') for n in convo if 'texter:' in n]
		messages_texter_all.append(len(messages_texter))
		messages_texter_join = ' '.join(messages_texter)
		word_count_texter.append(len(messages_texter_join.split()))

		messages_counselor = [n.replace('counselor: ','') for n in convo if 'counselor:' in n]
		messages_counselor_all.append(len(messages_counselor))
		messages_counselor_join = ' '.join(messages_counselor)
		word_count_counselor.append(len(messages_counselor_join.split()))
		
		convo_join = ' '.join(convo).replace('texter: ', '').replace('counselor: ', '')
		word_count_both.append(len(convo_join.split()))




	df[f'word_count'] = word_count_both
	df[f'word_count_texter'] = word_count_texter
	df[f'word_count_counselor'] = word_count_counselor
	df[f'word_count_texter_counselor_ratio'] = df[f'word_count_texter'] / df[f'word_count_counselor']
	df[f'amount_of_messages_texter'] = messages_texter_all
	df[f'amount_of_messages_counselor'] = messages_counselor_all
	df[f'amount_of_messages_texter_counselor_ratio'] = df[f'amount_of_messages_texter'] / df[f'amount_of_messages_counselor']



	return df 



train_subset = word_count_by_person(train_subset, 1000)
test_subset_25 = word_count_by_person(test_subset_25, messages_25)
test_subset_50 = word_count_by_person(test_subset_50, messages_50)
test_subset_75 = word_count_by_person(test_subset_75, messages_75)
test_subset = word_count_by_person(test_subset, 1000)



In [ ]:
# amount of tokens 

In [ ]:
test_subset_25['conversation_id'].values == test_subset['conversation_id'].values

In [ ]:
# train_subset['texter_word_count'] = train_subset['word_count'].values
# train_subset['texter_counselor_word_count'] = train_subset['word_count_with_interaction'].values


# test_subset['texter_word_count'] = test_subset['word_count'].values
# test_subset['texter_counselor_word_count'] = test_subset['word_count_with_interaction'].values






# # train_subset['counselor_word_count'] = train_subset['word_count_with_interaction'] - train_subset['word_count']

# # test_subset['texter_word_count'] = test_subset['word_count'].values
# # test_subset['counselor_word_count'] = test_subset['word_count_with_interaction'] - train_subset['word_count']



In [ ]:
# remove duplicate columns
for col in feature_names:
	try: train_subset = train_subset.drop(col+'_y',axis=1)
	except: pass

train_subset.columns = [n.replace('_x','') for n in train_subset.columns]


for col in feature_names:
	try: test_subset_50 = test_subset_50.drop(col+'_y',axis=1)
	except: pass

test_subset_50.columns = [n.replace('_x','') for n in test_subset_50.columns]



In [ ]:
train_subset

In [ ]:
test_subset_25

In [ ]:
from collections import Counter

X_train = train_subset[feature_names]
y_train = train_subset['dv'].values
X_test = test_subset[feature_names]
X_test_25 = test_subset_25[feature_names]
X_test_50 = test_subset_50[feature_names]
X_test_75 = test_subset_75[feature_names]

y_test = test_subset['dv'].values
print('train', Counter(y_train))
print('test',Counter(y_test))

test_subset.index = test_subset['conversation_id'].values
classes_i = classes['dv']
# balance test set
test_subset_1 = test_subset[test_subset['dv']==classes_i[1]]
test_subset_0 = test_subset[test_subset['dv']==classes_i[0]].sample(n=test_subset_1.shape[0],random_state=random_seed)
test_subset_balanced = pd.concat([test_subset_0, test_subset_1]).sample(frac=1).reset_index(drop=True)
X_test_balanced = test_subset_balanced[feature_names]
# TODO for 25,50,75 if you want 
X_test_balanced.index = test_subset_balanced['conversation_id'].values
y_test_balanced = test_subset_balanced['dv'].values
print('test',Counter(y_test_balanced))

In [ ]:
train_subset[train_subset['dv']=='active_rescue']['Active suicidal ideation & suicidal planning'].describe()

In [ ]:
test_subset[test_subset['dv']=='active_rescue']['Active suicidal ideation & suicidal planning'].describe()

In [ ]:
test_subset_25[test_subset_25['dv']=='active_rescue']['Active suicidal ideation & suicidal planning'].describe()

# Evaluate

In [ ]:
# if gridsearch == 'minority':
				# 	# Obtain all hyperparameter combinations
				# 	parameters = get_params(feature_vector,model_name=model_name, toy=toy)
				# 	parameter_set_combinations = get_combinations(parameters)
				# 	scores = {}
				# 	for i, set in enumerate(parameter_set_combinations):
				# 		pipeline.set_params(**set)
				# 		pipeline.fit(X_train,y_train)
				# 		y_pred = pipeline.predict(X_val) # validation set 
				# 		rmse_per_value = []
				# 		rmse = metrics.mean_squared_error(y_val, y_pred, squared=False ) # validation set 
				# 		for value in np.unique(y_val):
				# 			y_pred_test_i = [[pred,test] for pred,test in zip(y_pred,y_val) if test == value] # validation set 
				# 			y_pred_i = [n[0] for n in y_pred_test_i]
				# 			y_test_i = [n[1] for n in y_pred_test_i]
				# 			rmse_i = metrics.mean_squared_error(y_test_i, y_pred_i, squared=False )
				# 			rmse_per_value.append(rmse_i )
				# 		scores[i] = [rmse]+rmse_per_value+[str(set)]
				# 	scores = pd.DataFrame(scores).T
				# 	scores.columns = ['RMSE', 'RMSE_2', 'RMSE_3', 'RMSE_4', 'Parameters']
				# 	scores = scores.sort_values('RMSE_4')
				# 	best_params = eval(scores['Parameters'].values[0])
				# 	pipeline.set_params(**best_params)
				# 	pipeline.fit(X_train,y_train)
				# 	y_pred = pipeline.predict(X_test)

In [ ]:
reload(pipelines)

In [ ]:

from concept_tracker.utils import metrics_report
reload(metrics_report)
import warnings

toy = False
balance_test_set = False
random_seed = 123


sample_sizes = ['all']#[100, 500, 'all'] 
model_names = ['LGBMClassifier']# ['LogisticRegression']#,'LGBMClassifier'] #'XGBClassifier',
task = 'classification'
hyperparameter_tuning = 'bayesian'

if task == 'classification':
	scoring = 'f1'
	metrics_to_report = 'all'
	
elif task == 'regression':
	scoring = 'neg_mean_squared_error'
	metrics_to_report = 'all'

In [ ]:
dict(Counter(y_test))

In [ ]:
reload(metrics_report)

In [ ]:
reload(metrics_report)

In [ ]:
X_test_50

In [ ]:

if toy:
	sample_sizes = [50]
	feature_vectors = feature_vectors[:2]

ts_i = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

y_proba_1_all_models = []

for n in sample_sizes:
	if toy:
		output_dir_i = output_dir + f'results_{ts_i}_toy/'
	else:
		output_dir_i = output_dir + f'results_{ts_i}_{n}/'
	os.makedirs(output_dir_i, exist_ok=True)

	results = []
	# results_content_validity = []
	
	

	for feature_vector in feature_vectors:
		for dv in dvs:
			for amount_of_clauses in ['25%', '50%', '75%', '100%']:


				if balance_test_set:
					y_test_i = y_test_balanced.copy()
					X_test_i = X_test_balanced.copy()

				else:
					y_test_i = y_test.copy()
					if amount_of_clauses == '100%':
						X_test_i = X_test.copy()
					elif amount_of_clauses == '75%':
						
						X_test_i = X_test_75.copy()
					elif amount_of_clauses == '50%':
						
						X_test_i = X_test_50.copy()
					elif amount_of_clauses == '25%':
						
						X_test_i = X_test_25.copy()
					X_test_i = X_test_i.replace([np.inf, -np.inf], np.nan)
				

				classes_i = classes[dv]
				y_train_i = [int(n.replace(classes_i[1], '1').replace(classes_i[0], '0')) for n in y_train]
				y_test_i = [int(n.replace(classes_i[1], '1').replace(classes_i[0], '0')) for n in y_test_i]

				# if task == 'classification':
				# 	encoder = LabelEncoder()
				# 	# Fit and transform the labels to integers
				# 	y_train = encoder.fit_transform(y_train)
				# 	y_test = encoder.transform(y_test)

				# X_test_3_dv = X_test_3[X_test_3['y_test']==dv][feature_names]
				# y_test_3_dv = [1]*len(X_test_3_dv)

				if n !='all':
					X_train_i = X_train.copy()
					X_train_i['y'] = y_train_i
					X_train_i = X_train_i.sample(n = n)
					y_train_i = X_train_i['y'].values
					X_train_i = X_train_i.drop('y', axis=1)
				else:
					X_train_i = X_train.copy()
				if feature_vector != 'tfidf':
					if 'y' in X_test_i.columns or 'dv' in X_test_i.columns or classes_i[1] in X_test_i.columns:
						warnings.warn(f'y var is in X_test, skipping feature vector {feature_vector}')
						break
				

				for model_name in model_names: 

					pipeline = pipelines.get_pipelines(feature_vector, model_name = model_name, random_state=random_seed)
					print(pipeline)
				
					if hyperparameter_tuning == None or hyperparameter_tuning == False:
						pipeline.fit(X_train_i,y_train_i)
						best_params = 'No hyperparameter tuning'
					else:
						parameters = hyperparameters.get_params(feature_vector,model_name=model_name, toy=toy)
						pipeline, best_params = hyperparameters.hyparameter_tuning(pipeline, parameters, X_train_i, y_train_i, 
																	scoring = scoring,
																	method = 'bayesian', 
																	cv=cv, return_train_score=False,
																	n_iter=32, random_state=random_seed)
					
					# Performance
					dv_clean = dv.replace(' ','_').capitalize()
					if task == 'classification':
						y_proba = pipeline.predict_proba(X_test_i)       # Get predicted probabilities
						y_proba_1 = y_proba[:,1]
						y_proba_1_all_models.append(y_proba_1)
						y_pred = (y_proba_1>=0.5)*1                   # standard threshold
						output_filename = f'{feature_vector}_{model_name}_{classes_i[1]}_{n}_clauses-{amount_of_clauses}'
						custom_cr, sklearn_cr, cm_df_meaning, cm_df, cm_df_norm, y_pred_df = metrics_report.save_classification_performance(y_test_i, y_pred, y_proba_1, output_dir_i, output_filename=output_filename,feature_vector=feature_vector, model_name=model_name,best_params = best_params, classes = classes_i,amount_of_clauses=amount_of_clauses, save_output=True)

					
					# 	# results_i_content_3 = custom_classification_report(y_test_3_dv, y_pred_content_validity_3, y_pred_content_validity_3, output_dir_i,gridsearch=gridsearch,
					# 	# 						best_params=best_params,feature_vector=feature_vector,model_name=f'{feature_vector}_{model_name}_{dv}_content-validity-3',round_to = 2, ts = ts_i)
					# elif task == 'regression':
					# 	if gridsearch:
					# 		y_pred = best_model.predict(X_test)
					# 	else:
					# 		y_pred = pipeline.predict(X_test)

					# 	results_i =regression_report(y_test,y_pred,y_train=y_train,
					# 							metrics_to_report = metrics_to_report,
					# 								gridsearch=gridsearch,
					# 							best_params=best_params,feature_vector=feature_vector,model_name=model_name, plot = True, save_fig_path = path,n = n, round_to = 2)
					# # results_i.to_csv(output_dir_i + f'results_{feature_vector}_{model_name}_gridsearch-{gridsearch}_{n}_{ts_i}_{dv}.csv')
					display(custom_cr)
					results.append(custom_cr)
					# results_content_validity.append(results_i_content_13)
					# results_content_validity.append(results_i_content_3)
					# Feature importance
					# if feature_vector == 'tfidf':
					# 	if model_name in ['XGBRegressor']:
					# 		warnings.warn('Need to add code to parse XGBoost feature importance dict')
					# 	else:
					# 		feature_importances = tfidf_feature_importances(pipeline, top_k = 50, savefig_path = output_dir_i + f'feature_importance_{feature_vector}_{model_name}_{n}_{ts_i}_{dv}')
					# else:
					feature_names = X_train_i.columns
					# TODO add correlation with DV to know direction
					feature_importance = metrics_report.generate_feature_importance_df(pipeline, model_name,feature_names,  model_name_in_pipeline='model')
					if str(feature_importance) != 'None':       # I only implemented a few methods for a few models
						feature_importance.to_csv(output_dir_i + f'feature_importance_{output_filename}.csv', index = False)        
						# display(feature_importance.iloc[:50])
					
	results_df = pd.concat(results)
	results_df = results_df.reset_index(drop=True)
	output_filename = f'{feature_vector}_{classes_i[1]}_{n}'

	results_df.to_csv(output_dir_i + f'results_{output_filename}.csv', index=False)

	# results_df_content_validity = pd.concat(results_content_validity)
	# results_df_content_validity = results_df_content_validity.reset_index(drop=True)
	# results_df_content_validity.to_csv(output_dir_i + f'results_content_validity_{n}_{ts_i}.csv', index=False)


	

	# NaN analysis
	if type(X_train_i) == pd.core.frame.DataFrame:
		df = X_train_i.copy()
		# Find the column and index of NaN values
		nan_indices = df.index[df.isnull().any(axis=1)].tolist()
		nan_columns = df.columns[df.isnull().any()].tolist()
		# print("Indices of NaN values:", nan_indices)
		print("Columns with NaN values:", nan_columns)
		print(df.size)
		nans = df.isna().sum().sum()
		print('% of nans:', np.round(nans/df.size,3))
	




In [ ]:



infinite_rows = np.isinf(X_test_i).any(axis=1)
print("Rows with infinite values:")
print(X_test_i[infinite_rows].index)


# X = X.replace([np.inf, -np.inf], np.nan)


In [ ]:


results_filename = f'results_cts_51_srl_prototypes_active_rescue_all.csv'
results_dir = f'results_24-03-22T18-05-48_all_LGBMClassifier/'

results_df = pd.read_csv('./data/output/active_rescue/' + results_dir + results_filename)
results_df_binary = results_df[(results_df['Class']==1) & (results_df['Amount of clauses']=='binary')]
results_df_binary.rename(columns = {'Average':f'% of messages'}, inplace = True)
results_df_binary.rename(columns = {'Amount of clauses':'Average'}, inplace = True)

# plot sensitivity, specificity, precision, ROC AUC, PR AUC
results_df_binary
# use lines with dots 

sns.lineplot(data=	results_df_binary, y="Sensitivity", x=f"% of messages", label ='Sensitivity')
sns.lineplot(data=	results_df_binary, y="Specificity", x=f"% of messages", label = 'Specificity')
sns.lineplot(data=	results_df_binary, y="Precision", x=f"% of messages",	label = 'Precision')
sns.lineplot(data=	results_df_binary, y="ROC AUC", x=f"% of messages",	label = 'ROC AUC')
sns.lineplot(data=	results_df_binary, y="PR AUC", x=f"% of messages",	label = 'PR AUC', color = 'pink')
# horizontal line

plt.axhline(y=0.22, xmin = 0.04,xmax=0.96, linestyle='--', label = 'Baseline for PR AUC\nall rescues (P/N)',  color = 'pink')
plt.ylabel('Prediction of active rescue')
# place legend outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)



In [ ]:
import pandas as pd
from sklearn import metrics

def  nrui_score(net_benefit, net_benefit_treat_all, threshold):
	#  net reduction in unnecessary interventions 
	nrui = 100* (net_benefit - net_benefit_treat_all) / (threshold / (1 - threshold))
	return nrui


# Assuming the true positive, false positive rates and pt are provided or computed beforehand
# This function calculates the net benefit
def calculate_net_benefit(tp, fp, N, pt):
	net_benefit = (tp/N) - (fp / N) * (pt / (1 - pt))
	return net_benefit



for i, amount_of_clauses, model_name in zip(range(4), ['25%', '50%', '75%', '100%'], ['LGBMClassifier', 'LGBMClassifier', 'LGBMClassifier', 'LGBMClassifier']):
	y_proba_1 = y_proba_1_all_models[i]


	# compute precision and recall for each threshold
	thresholds = np.arange(0.005, 0.2, 0.005)
	net_benefits = []
	net_benefits_all_1 = []
	net_benefits_all_0 = []


	plt.clf()
	sns.set(rc={'figure.figsize':(4,4)}) #w
	sns.set_style("white")
	nruis = []
	nruis_all_0 = []

	for threshold in thresholds:
		N = len(y_proba_1)
		tn, fp, fn, tp = metrics.confusion_matrix(y_test_i, y_proba_1 > threshold).ravel()
		net_benefit = calculate_net_benefit(tp, fp, N, threshold)
		net_benefits.append(net_benefit)

		tn_all_1, fp_all_1, fn_all_1, tp_all_1 = metrics.confusion_matrix(y_test_i, y_proba_1 > 0).ravel()
		net_benefit_all_1 = calculate_net_benefit(tp_all_1, fp_all_1, N, threshold)
		net_benefits_all_1.append(net_benefit_all_1)

		tn_all_0, fp_all_0, fn_all_0, tp_all_0 = metrics.confusion_matrix(y_test_i, y_proba_1 > 1).ravel()
		net_benefit_all_0 = calculate_net_benefit(tp_all_0, fp_all_0, N, threshold)
		net_benefits_all_0.append(net_benefit_all_0)

		nrui = nrui_score(net_benefit, net_benefit_all_1, threshold)
		nrui_all_0 = nrui_score(net_benefit_all_0, net_benefit_all_1, threshold)

		nruis.append(nrui)
		nruis_all_0.append(nrui_all_0)

	plt.clf()
	sns.set(rc={'figure.figsize':(4,4)})
	sns.set_style("white")
	thresholds_perc = [f'{np.round(n * 100, 1)}' for n in thresholds]
	plt.title(amount_of_clauses)
	plt.plot(thresholds, net_benefits, label = model_name)
	plt.plot(thresholds, net_benefits_all_1, label = 'Rescue all', color = 'pink')
	plt.plot(thresholds, net_benefits_all_0, label = 'Rescue None')
	# display xticklabels, which are proportions, as probabilities (multiply by 100)
	skip  = 6
	plt.xticks(thresholds[::skip], thresholds_perc[::skip])
	# plt.title('Net benefit')
	plt.xlabel('Threshold probability')
	plt.ylabel('Net Benefit')
	plt.legend()
	# plot horizontal line at 0
	# plt.axhline(y=0, color='gray', linestyle='--')
	plt.show()

	plt.clf()
	sns.set(rc={'figure.figsize':(4,4)})
	sns.set_style("white")
	plt.title(amount_of_clauses)
	plt.plot(thresholds, nruis, label = model_name)
	# plt.plot(thresholds, nruis_all_0, label = 'Rescue None')
	plt.xticks(thresholds[::skip], thresholds_perc[::skip])
	plt.xlabel('Threshold probability')
	# make ylabel font smaller 

	plt.ylabel('Net reduction in interventions\nper 100 individuals') # net_benefits_all_0
	plt.tight_layout()
	plt.show()







In [ ]:
results_filename = f'results_cts_51_srl_prototypes_active_rescue_all.csv'
results_dir = f'results_24-03-22T17-32-20_all/'

results_df = pd.read_csv('./data/output/active_rescue/' + results_dir + results_filename)
results_df_binary = results_df[(results_df['Class']==1) & (results_df['Amount of clauses']=='binary')]
results_df_binary.rename(columns = {'Average':f'% of messages'}, inplace = True)
results_df_binary.rename(columns = {'Amount of clauses':'Average'}, inplace = True)

# plot sensitivity, specificity, precision, ROC AUC, PR AUC
results_df_binary
# use lines with dots 

sns.lineplot(data=	results_df_binary, y="Sensitivity", x=f"% of messages", label ='Sensitivity')
sns.lineplot(data=	results_df_binary, y="Specificity", x=f"% of messages", label = 'Specificity')
sns.lineplot(data=	results_df_binary, y="Precision", x=f"% of messages",	label = 'Precision')
sns.lineplot(data=	results_df_binary, y="ROC AUC", x=f"% of messages",	label = 'ROC AUC')
sns.lineplot(data=	results_df_binary, y="PR AUC", x=f"% of messages",	label = 'PR AUC', color = 'pink')
# horizontal line

plt.axhline(y=0.22, xmin = 0.04,xmax=0.96, linestyle='--', label = 'Baseline for PR AUC\nall rescues (P/N)',  color = 'pink')
plt.ylabel('Prediction of active rescue')
# place legend outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)



In [ ]:
import pandas as pd
from sklearn import metrics

def  nrui_score(net_benefit, net_benefit_treat_all, threshold):
	#  net reduction in unnecessary interventions 
	nrui = 100* (net_benefit - net_benefit_treat_all) / (threshold / (1 - threshold))
	return nrui


# Assuming the true positive, false positive rates and pt are provided or computed beforehand
# This function calculates the net benefit
def calculate_net_benefit(tp, fp, N, pt):
	net_benefit = (tp/N) - (fp / N) * (pt / (1 - pt))
	return net_benefit



for i, amount_of_clauses, model_name in zip(range(4), ['25%', '50%', '75%', '100%'], ['LogisticRegression', 'LogisticRegression', 'LogisticRegression', 'LogisticRegression']):
	y_proba_1 = y_proba_1_all_models[i]


	# compute precision and recall for each threshold
	thresholds = np.arange(0.005, 0.2, 0.005)
	net_benefits = []
	net_benefits_all_1 = []
	net_benefits_all_0 = []


	plt.clf()
	sns.set(rc={'figure.figsize':(4,4)}) #w
	sns.set_style("white")
	nruis = []
	nruis_all_0 = []

	for threshold in thresholds:
		N = len(y_proba_1)
		tn, fp, fn, tp = metrics.confusion_matrix(y_test_i, y_proba_1 > threshold).ravel()
		net_benefit = calculate_net_benefit(tp, fp, N, threshold)
		net_benefits.append(net_benefit)

		tn_all_1, fp_all_1, fn_all_1, tp_all_1 = metrics.confusion_matrix(y_test_i, y_proba_1 > 0).ravel()
		net_benefit_all_1 = calculate_net_benefit(tp_all_1, fp_all_1, N, threshold)
		net_benefits_all_1.append(net_benefit_all_1)

		tn_all_0, fp_all_0, fn_all_0, tp_all_0 = metrics.confusion_matrix(y_test_i, y_proba_1 > 1).ravel()
		net_benefit_all_0 = calculate_net_benefit(tp_all_0, fp_all_0, N, threshold)
		net_benefits_all_0.append(net_benefit_all_0)

		nrui = nrui_score(net_benefit, net_benefit_all_1, threshold)
		nrui_all_0 = nrui_score(net_benefit_all_0, net_benefit_all_1, threshold)

		nruis.append(nrui)
		nruis_all_0.append(nrui_all_0)

	plt.clf()
	sns.set(rc={'figure.figsize':(4,4)})
	sns.set_style("white")
	thresholds_perc = [f'{np.round(n * 100, 1)}' for n in thresholds]
	plt.title(amount_of_clauses)
	plt.plot(thresholds, net_benefits, label = model_name)
	plt.plot(thresholds, net_benefits_all_1, label = 'Rescue all', color = 'pink')
	plt.plot(thresholds, net_benefits_all_0, label = 'Rescue None')
	# display xticklabels, which are proportions, as probabilities (multiply by 100)
	skip  = 6
	plt.xticks(thresholds[::skip], thresholds_perc[::skip])
	# plt.title('Net benefit')
	plt.xlabel('Threshold probability')
	plt.ylabel('Net Benefit')
	plt.legend()
	# plot horizontal line at 0
	# plt.axhline(y=0, color='gray', linestyle='--')
	plt.show()

	plt.clf()
	sns.set(rc={'figure.figsize':(4.5,4)})
	sns.set_style("white")
	plt.title(amount_of_clauses)
	plt.plot(thresholds, nruis, label = model_name)
	# plt.plot(thresholds, nruis_all_0, label = 'Rescue None')
	plt.xticks(thresholds[::skip], thresholds_perc[::skip])
	plt.xlabel('Threshold probability')
	# make ylabel font smaller 

	plt.ylabel('Net reduction in interventions\nper 100 individuals') # net_benefits_all_0
	plt.tight_layout()
	plt.show()







In [ ]:
from sklearn import metrics

In [ ]:

y_pred_opt = (y_proba_1>=0.847)*1                   # standard threshold
y_pred_opt

custom_cr, sklearn_cr, cm_df_meaning, cm_df, cm_df_norm, y_pred_df = metrics_report.save_classification_performance(y_test_i, y_pred_opt, y_proba_1, output_dir_i, output_filename=output_filename,feature_vector=feature_vector, model_name=model_name,best_params = best_params, classes = classes_i, save_output=False)
print(y_test_i, y_pred)
display(custom_cr)
display(sklearn_cr)

In [ ]:
0.2*0.22

In [ ]:
np.sum(y_pred)/len(y_pred)

In [ ]:
import pandas as pd


def  nrui_score(net_benefit, net_benefit_treat_all, threshold):
	#  net reduction in unnecessary interventions 
	nrui = 100* (net_benefit - net_benefit_treat_all) / (threshold / (1 - threshold))
	return nrui


# Assuming the true positive, false positive rates and pt are provided or computed beforehand
# This function calculates the net benefit
def calculate_net_benefit(tp, fp, N, pt):
	net_benefit = (tp/N) - (fp / N) * (pt / (1 - pt))
	return net_benefit

# compute precision and recall for each threshold
thresholds = np.arange(0.005, 0.2, 0.005)
net_benefits = []
net_benefits_all_1 = []
net_benefits_all_0 = []


plt.clf()
sns.set(rc={'figure.figsize':(4,4)}) #w
sns.set_style("white")
nruis = []
nruis_all_0 = []

for threshold in thresholds:
	N = len(y_proba_1)
	tn, fp, fn, tp = metrics.confusion_matrix(y_test_i, y_proba_1 > threshold).ravel()
	net_benefit = calculate_net_benefit(tp, fp, N, threshold)
	net_benefits.append(net_benefit)

	tn_all_1, fp_all_1, fn_all_1, tp_all_1 = metrics.confusion_matrix(y_test_i, y_proba_1 > 0).ravel()
	net_benefit_all_1 = calculate_net_benefit(tp_all_1, fp_all_1, N, threshold)
	net_benefits_all_1.append(net_benefit_all_1)

	tn_all_0, fp_all_0, fn_all_0, tp_all_0 = metrics.confusion_matrix(y_test_i, y_proba_1 > 1).ravel()
	net_benefit_all_0 = calculate_net_benefit(tp_all_0, fp_all_0, N, threshold)
	net_benefits_all_0.append(net_benefit_all_0)

	nrui = nrui_score(net_benefit, net_benefit_all_1, threshold)
	nrui_all_0 = nrui_score(net_benefit_all_0, net_benefit_all_1, threshold)

	nruis.append(nrui)
	nruis_all_0.append(nrui_all_0)

plt.clf()
sns.set(rc={'figure.figsize':(4,4)})
sns.set_style("white")
thresholds_perc = [f'{np.round(n * 100, 1)}' for n in thresholds]
plt.plot(thresholds, net_benefits, label = 'LGBMClassifier')
plt.plot(thresholds, net_benefits_all_1, label = 'Rescue all')
plt.plot(thresholds, net_benefits_all_0, label = 'Rescue None')
# display xticklabels, which are proportions, as probabilities (multiply by 100)
skip  = 6
plt.xticks(thresholds[::skip], thresholds_perc[::skip])
# plt.title('Net benefit')
plt.xlabel('Threshold probability')
plt.ylabel('Net Benefit')
plt.legend()
# plot horizontal line at 0
# plt.axhline(y=0, color='gray', linestyle='--')
plt.show()

plt.clf()
sns.set(rc={'figure.figsize':(4,4)})
sns.set_style("white")
plt.plot(thresholds, nruis, label = 'LGBMClassifier')
# plt.plot(thresholds, nruis_all_0, label = 'Rescue None')
plt.xticks(thresholds[::skip], thresholds_perc[::skip])
plt.xlabel('Threshold probability')
# make ylabel font smaller 

plt.ylabel('Net reduction in interventions\nper 100 individuals', fontsize = 10) # net_benefits_all_0
plt.tight_layout()
plt.show()







In [ ]:
def  nrui_score(net_benefit, net_benefit_treat_all, threshold):
	#  net reduction in unnecessary interventions 
	nrui = 100* (net_benefit - net_benefit_treat_all) / (threshold / (1 - threshold))
	return nrui

In [ ]:
thresholds

In [ ]:
net_benefits

In [ ]:
for feature in X_train

In [ ]:
dict(Counter(y_true)).values()

In [ ]:
np.sum(list(dict(Counter(y_true)).values()))

In [ ]:
# !pip install lofo-importance


In [ ]:
!pip install pymc3 

In [ ]:
np.__version__

In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

# Sample data
np.random.seed(42)
X = np.linspace(0, 10, 100)[:, None]
y = 2.5 * X.squeeze() + np.random.normal(scale=1.0, size=100)

# Bayesian linear regression model
with pm.Model() as model:
    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Expected value of outcome
    mu = alpha + beta * X

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=y)

    # Posterior distribution and sampling
    trace = pm.sample(1000, return_inferencedata=False)

# Posterior predictive checks to obtain predictions and uncertainties
with model:
    posterior_pred = pm.sample_posterior_predictive(trace)

# Extract predicted values and uncertainties
predictions = posterior_pred['Y_obs'].mean(axis=0)
uncertainties = posterior_pred['Y_obs'].std(axis=0)

# Plot the results
plt.scatter(X, y, label='Observed data')
plt.plot(X, predictions, label='Predicted mean')
plt.fill_between(X.squeeze(), predictions - uncertainties, predictions + uncertainties, alpha=0.2, label='Uncertainty interval')
plt.legend()
plt.show()

print(predictions)
print(uncertainties)


In [ ]:
from lofo import LOFOImportance, Dataset, plot_importance


In [ ]:
pipeline

In [ ]:
# X_train_i['active_rescue'] = y_train_i    
# dataset = Dataset(df=X_train_i, target="active_rescue", features=[col for col in X_train_i.columns if col != "active_rescue"], 
# 				  auto_group_threshold=0.8)

# lofo_imp = LOFOImportance(dataset, cv=3, scoring=scoring, model=pipeline)
# lofo_imp.get_importance()

In [ ]:

# def get_lofo_importance(target):
#     cv = KFold(n_splits=7, shuffle=True, random_state=17)

#     dataset = Dataset(df=df[df[target].notnull()], target=target, features=loading_features,
#                       feature_groups={"fnc": df[df[target].notnull()][fnc_features].values
#                       })

#     model = Ridge(alpha=0.01)
#     lofo_imp = LOFOImportance(X, cv=cv, scoring="f1", model=pipeline)

#     return lofo_imp.get_importance()

# plot_importance(get_lofo_importance(target="domain1_var1"), figsize=(8, 8), kind="box")

# Explainability: construct tokens vs. doc tokens


In [ ]:
with open('./data/input/ctl/embeddings/'+'X_test_all_with_interaction_desire_active_rescue_subset_tokenized_clauses_cts-prototypes_cosine_similarities.pickle', 'rb') as handle:
	cosine_scores_per_doc = dill.load(handle)
	# keys are '1508885_Passive suicidal ideation'




In [ ]:

srl = dill.load(open("./../lexicon/data/input/lexicons/suicide_risk_lexicon_validated_prototypical_tokens_24-03-06T00-47-30.pickle", "rb"))
constructs_to_measure = srl_constructs.constructs_in_order

construct_tokens_d = {}
for construct in srl.constructs.keys():
	tokens = srl.constructs[construct]['tokens']                      
	construct_tokens_d[construct] = tokens

In [ ]:
def return_cosine_df(doc_id, construct, docs_clauses, construct_tokens_d, cosine_scores_per_doc):
  doc_clauses_i = docs_clauses[doc_id]
  construct_tokens_i = construct_tokens_d[construct]
  df = pd.DataFrame(cosine_scores_per_doc[f'{doc_id}_{construct}'], index = construct_tokens_i, columns = doc_clauses_i)
  return df


In [ ]:
docs_clauses = dict(zip(test_subset['conversation_id'].values, test_subset['message_with_interaction_clean_clauses'].values))

In [ ]:
list(cosine_scores_per_doc.keys())[0]


In [ ]:
doc_clauses_i = docs_clauses[convo_id]
doc_clauses_i

In [ ]:
construct_tokens_i = construct_tokens_d[construct]
construct_tokens_i

In [ ]:
cosine_scores_per_doc[f'{convo_id}_{construct}'].shape

In [ ]:
eval(doc_clauses_i)

In [ ]:
df = pd.DataFrame(cosine_similarities, index = construct_tokens_i, columns = eval(doc_clauses_i))

In [ ]:
# Return index, column of highest value
df.idxmax(axis=1), df.idxmax(axis=0)


In [ ]:
def highest_message(convo_id, construct, docs_clauses, construct_tokens_d, cosine_scores_per_doc):
	cosine_similarities = cosine_scores_per_doc[str(convo_id)+'_'+construct].copy()
	doc_clauses_i = eval(docs_clauses[convo_id])
	construct_tokens_i = construct_tokens_d[construct]
	df = pd.DataFrame(cosine_similarities, index = construct_tokens_i, columns = doc_clauses_i)
	max_value = df.max().max()
	# Find the column and index of the maximum value
	column = df.max().idxmax()
	index = df[column].idxmax()
	# Return [value, column, index]
	max_col_index = [max_value, column, index]
	return df, max_col_index
	

In [ ]:
construct_examples = []


for construct in srl_constructs.constructs_in_order:
  high_scoring_docs_convo_ids = X_test_i[construct].nlargest(20).index.tolist()
  for convo_id in high_scoring_docs_convo_ids:
	df, max_col_index = highest_message(convo_id, construct, docs_clauses, construct_tokens_d, cosine_scores_per_doc)
	construct_examples.append([construct] + max_col_index)
	
construct_examples_df = pd.DataFrame(construct_examples, columns = ['Construct', 'Value', 'Message', 'Construct token'])
# construct_examples_df.round(2).
construct_examples_df['temp_message_token'] = construct_examples_df['Message']+' '+construct_examples_df['Construct token']
construct_examples_df['temp_message_token'] = construct_examples_df['temp_message_token'].str.lower()
construct_examples_df = construct_examples_df.drop_duplicates(subset=['temp_message_token'])
construct_examples_df = construct_examples_df.drop(columns = ['temp_message_token'])
construct_examples_df = construct_examples_df[construct_examples_df['Value']>0.55]
construct_examples_df = construct_examples_df.round(2)
construct_examples_df

	
construct_examples_df.to_csv('./data/output/active_rescue/construct_examples.csv', index = False)
  

  
  

In [ ]:

'''
construct = 'gratitude'
doc_id = 65

construct = 'anger'
doc_id = 22

construct = 'annoyance'
doc_id = 64
'''

construct = 'gratitude'
doc_id = 65



df = return_cosine_df(doc_id, construct, docs_clauses, construct_tokens_d,X_test_cosine_scores_per_doc)
display(df)